# Baseline Model and Pipeline for Hit-Predict Project

## Table of Contents
1. [Setup](#setup)
    - [Importing Libraries](#lib)
    - [Loading Data](#data)
2. [Baseline Models](#baseline)
    - [Regression Problem](#sub1)
    - [Classification Problem with Classes by 20](#sub2)
    - [Classification Problem with Classes by 5](#sub3)
3. [Final Model Pipeline](#pipeline)

## 1. Setup
<a id="setup"></a>

### Importing Libraries
<a id="lib"></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import silhouette_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor

from scipy.stats import zscore

### Loading Data
<a id="data"></a>

In [3]:
DATA_PATH = "../data/processed_spotify_songs.csv"
df = pd.read_csv(DATA_PATH)

## 2. Baseline Models
<a id="baseline"></a>

Recall that in the EDA notebook, we noticed that there was a high class imbalance in the target variable, especially with regards to a very high number of tracks that had a popularity score of 0. We discussed different ways to address this imbalance in the preprocessing steps of our data in previous notebooks and saved the different final preprocessed data in 3 different csv files. We will thus test all of our methods against these 3 preprocessing methods to evaluate potential improvements or not of the performances of our different models.

## Regression Problem

## Classification Problem with Classes by 20

## Classification Problem with Classes by 5